In [0]:
!pip install torch torchvision
!pip install -U finance-datareader


In [0]:
import FinanceDataReader as fdr
import datetime
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import numpy as np
import argparse
import time
from copy import deepcopy # Add Deepcopy for args
from sklearn.metrics import mean_absolute_error
import FinanceDataReader as fdr

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
class StockDataset(Dataset):
    
    def __init__(self, symbol, x_frames, y_frames, start, end):
        
        self.symbol = symbol
        self.x_frames = x_frames
        self.y_frames = y_frames
        
        self.start = datetime.datetime(*start)
        self.end = datetime.datetime(*end)

        self.data = fdr.DataReader(self.symbol, self.start, self.end)
        print(self.data.isna().sum())
        
    def __len__(self):
        return len(self.data) - (self.x_frames + self.y_frames) + 1
    
    def __getitem__(self, idx):
        idx += self.x_frames
        data = self.data.iloc[idx-self.x_frames:idx+self.y_frames]
        data = data[['High', 'Low', 'Open', 'Close', 'Change', 'Volume']]
        data = data.apply(lambda x: (x+1) / (x[self.x_frames-1]+1))
        data = data.values
        X = data[:self.x_frames]
        y = data[self.x_frames:]
        
        return X, y

In [0]:
class TestDataset(Dataset):
    
    def __init__(self, symbol, x_frames, y_frames):
        
        self.symbol = symbol
        self.x_frames = x_frames
        self.y_frames = y_frames
        
        self.start = datetime.datetime.now() - datetime.timedelta(days=10)
        self.end = datetime.date.today()
        self.data = fdr.DataReader(self.symbol, self.start, self.end)
        self.data = self.data.tail(6)
        print(self.data.isna().sum())
        
    def __len__(self):
        return len(self.data) - (self.x_frames + self.y_frames) + 1
    
    def __getitem__(self, idx):
        idx += self.x_frames
        data = self.data.iloc[idx-self.x_frames:idx+self.y_frames]
        data = data[['High', 'Low', 'Open', 'Close', 'Change', 'Volume']]
        #data = data.apply(lambda x: (x+1) / (x[self.x_frames-1]+1))
        data = data.values
        X = data[:self.x_frames]
        y = data[self.x_frames:]
        
        return X, y

In [0]:
class PredictDataset(Dataset):
    
    def __init__(self, symbol, x_frames):
        
        self.symbol = symbol
        self.x_frames = x_frames
        
        self.start = datetime.datetime.now() - datetime.timedelta(days=10)
        self.end = datetime.date.today()
        self.data = fdr.DataReader(self.symbol, self.start, self.end)
        self.data = self.data.tail(3)
        print(self.data.isna().sum())
        
    def __len__(self):
        return len(self.data) - (self.x_frames) + 1
    
    def __getitem__(self, idx):
        idx += self.x_frames
        data = self.data.iloc[idx-self.x_frames:idx]
        data = data[['High', 'Low', 'Open', 'Close', 'Change', 'Volume']]
        #data = data.apply(lambda x: (x+1) / (x[self.x_frames-1]+1))
        data = data.values
        X = data[:self.x_frames]
        
        return X

In [0]:
class LSTM(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, batch_size):
        super(LSTM, self).__init__()
        self.input_dim = input_dim 
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers

        self.batch_size = batch_size
        
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers)
        self.hidden = self.init_hidden()
        self.regressor = self.make_regressor()
        
    def init_hidden(self):
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))
    
    def make_regressor(self):
        layers = []
        
        layers.append(nn.Linear(self.hidden_dim, self.hidden_dim // 2))
        layers.append(nn.ReLU())
        layers.append(nn.Linear(self.hidden_dim // 2, self.output_dim))
        regressor = nn.Sequential(*layers)
        return regressor
    
    def forward(self, x):
        lstm_out, self.hidden = self.lstm(x, self.hidden)
        y_pred = self.regressor(lstm_out[-1].view(self.batch_size, -1))
        return y_pred

In [0]:
#테스트 데이터 생성
symbols = "001040,001450,024110,020150,102110,071050,086790,003670,032830,010950,005830,005940,180640,081660,003410,004020,005490,000120,051900,002790,122630,047810,011170,128940,005380,078930,011780,021240,036460,051910,006800,035420,034220,097950,011070,008770,000210,251270,000080,000720,018880,011790,005387,030200,018260,033780,008560,153130,012510,271560,012330,035720,015760,003550,028260,161390,028050,029780,007070,009540,069500,036570,009150,096770,090430,005930,005935,241560,105560,252670,000270,032640,004170,086280,012750,000660,010130,267250,004990,066570,009830,282330,023530,000810,316140,016360,139480,009240,017670,088980,034730,006400,000100,207940,006360,055550,035250,010140,007310,068270".split(",")

In [0]:
batch_size = 1
input_dim = 6
hidden_dim = 50
output_dim = 3
num_layers = 1

model = LSTM(input_dim,hidden_dim,output_dim,num_layers,batch_size)
modelPath = '/content/gdrive/My Drive/LSTM_batch_1_model.pt'
model.load_state_dict(torch.load(modelPath))
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model.to(device)

LSTM(
  (lstm): LSTM(6, 50)
  (regressor): Sequential(
    (0): Linear(in_features=50, out_features=25, bias=True)
    (1): ReLU()
    (2): Linear(in_features=25, out_features=3, bias=True)
  )
)

In [0]:
result = {}

In [0]:
for sb in symbols:
    testset = TestDataset(sb,3,3)
    
    model.eval()
    plr = DataLoader(testset,batch_size,shuffle=True)

    for (X,y) in plr:
        X = X.transpose(0, 1).float().to(device)
        y_true = y[:, :, 3].float().to(device)
        model.hidden = [hidden.to(device) for hidden in model.init_hidden()]
        y_pred = model(X)

    result[sb] = y_pred[0][2]*y_true[0][2]
    print(y_pred[0][2]*y_true[0][2])


2020-05-11 01:20:11.388885
2020-05-21
             Open   High    Low  Close  Volume    Change
Date                                                    
2020-05-14  79300  81000  78700  79300   98462  0.003797
2020-05-15  80400  82000  79400  80000  117090  0.008827
2020-05-18  81700  85400  80600  82300  195675  0.028750
2020-05-19  84300  84700  83000  84300  153536  0.024301
2020-05-20  84700  87200  84500  86500  158284  0.026097
2020-05-21  86500  87000  85500  86400   31108 -0.001156
Open      0
High      0
Low       0
Close     0
Volume    0
Change    0
dtype: int64
tensor(87631.9531, device='cuda:0', grad_fn=<MulBackward0>)
2020-05-11 01:20:14.093097
2020-05-21
             Open   High    Low  Close  Volume    Change
Date                                                    
2020-05-14  26800  26800  26000  26150  415251 -0.020599
2020-05-15  26800  27000  26400  26700  500026  0.021033
2020-05-18  26900  26900  25500  25500  487550 -0.044944
2020-05-19  26000  26500  24850  24950

In [0]:
print(result)

{'001040': tensor(87631.9531, device='cuda:0', grad_fn=<MulBackward0>), '001450': tensor(25102.9043, device='cuda:0', grad_fn=<MulBackward0>), '024110': tensor(7809.7925, device='cuda:0', grad_fn=<MulBackward0>), '020150': tensor(48684.4180, device='cuda:0', grad_fn=<MulBackward0>), '102110': tensor(26817.0020, device='cuda:0', grad_fn=<MulBackward0>), '071050': tensor(51017.2148, device='cuda:0', grad_fn=<MulBackward0>), '086790': tensor(26522.8652, device='cuda:0', grad_fn=<MulBackward0>), '003670': tensor(59739.8398, device='cuda:0', grad_fn=<MulBackward0>), '032830': tensor(46605.1875, device='cuda:0', grad_fn=<MulBackward0>), '010950': tensor(73533.7578, device='cuda:0', grad_fn=<MulBackward0>), '005830': tensor(47467.3086, device='cuda:0', grad_fn=<MulBackward0>), '005940': tensor(9929.5928, device='cuda:0', grad_fn=<MulBackward0>), '180640': tensor(80329.2891, device='cuda:0', grad_fn=<MulBackward0>), '081660': tensor(37223.2969, device='cuda:0', grad_fn=<MulBackward0>), '003410

In [0]:
ok = fdr.DataReader('001040')

ok.tail(10)

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2020-05-08,84800,84900,79500,80000,151176,-0.038462
2020-05-11,80100,80500,78700,78800,90846,-0.015000
2020-05-12,79200,79300,76600,77300,74619,-0.019036
2020-05-13,77300,79400,76300,79000,79824,0.021992
2020-05-14,79300,81000,78700,79300,98462,0.003797
2020-05-15,80400,82000,79400,80000,117090,0.008827
2020-05-18,81700,85400,80600,82300,195675,0.028750
2020-05-19,84300,84700,83000,84300,153536,0.024301
2020-05-20,84700,87200,84500,86500,158284,0.026097


In [0]:
pr = PredictDataset('001040',3)

xx = DataLoader(pr,1,False)

for X in xx:
    X = X.transpose(0, 1).float().to(device)
    Xtos = X[:,:,3].view(-1)
    print(Xtos)
    model.hidden = [hidden.to(device) for hidden in model.init_hidden()]
    y_pred = model(X)
    print(y_pred.view(-1)*Xtos)

Open      0
High      0
Low       0
Close     0
Volume    0
Change    0
dtype: int64
tensor([84300., 86500., 86500.], device='cuda:0')
tensor([86203.8281, 87339.8203, 87733.3828], device='cuda:0',
       grad_fn=<MulBackward0>)
